## install requirment and venv

In [1]:
# create venv for this chapter
!python3 -m venv chapter_env

In [2]:
#actiavate it 
!source chapter_env/bin/activate

In [3]:
!python3 -m pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable


#### importing all lib for this chapter

In [74]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage , ToolMessage
from langchain_core.tools import tool
from langchain_core.prompts.chat import ChatPromptTemplate , MessagesPlaceholder 
from langchain_core.runnables.history import RunnableWithMessageHistory 
from langchain_community.chat_message_histories import ChatMessageHistory
import os, dotenv , tiktoken

In [5]:
dotenv.load_dotenv()
google_api_key = os.getenv("GOOGLE_API_KEY") or ""
openai_api_key = os.getenv("OPENAI_API_KEY") or ""

#### simple usage of chat prompt template

In [6]:
template = ChatPromptTemplate(
    messages=[
        ("system","You are a helpful AI bot. Your name is {name}."),
        ("human","{user_input}")
    ]
)

In [7]:
prompt_value = template.invoke(
    {
        "name":"AI From Ollama",
        "user_input":"hello , what is your name"
    }
)

In [8]:
print(prompt_value.messages)

[SystemMessage(content='You are a helpful AI bot. Your name is AI From Ollama.', additional_kwargs={}, response_metadata={}), HumanMessage(content='hello , what is your name', additional_kwargs={}, response_metadata={})]


In [9]:
for msg in prompt_value.messages:
    print(msg , end="\n")

content='You are a helpful AI bot. Your name is AI From Ollama.' additional_kwargs={} response_metadata={}
content='hello , what is your name' additional_kwargs={} response_metadata={}


#### now work with this in ollama langchain

In [10]:
llm = ChatOllama(
    model="gemma3:270m",
    timeout=30,
    base_url="http://127.0.0.1:11434",
    use_mmap=True,
)

In [11]:
chain = template | llm

In [12]:
response = chain.invoke({
    "name": "AI From Ollama",
    "user_input": "hello, what is your name?"
})

print(response.content)

I am Gemma, an open-weights AI model created by the Gemma team.


#### now add memory to the chat

In [13]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI bot. Your name is {name}."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{user_input}")
])

In [14]:
store = {}

In [15]:
def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [16]:
chain_with_memory = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="user_input",
    history_messages_key="history",
)

In [17]:
response1 = chain_with_memory.invoke(
    {"name": "AI From Ollama", "user_input": "Hello!, my name is zkzk"},
    config={"configurable": {"session_id": "user1"}}
)

print(response1.content)


Hello! I'm zkzk, your friendly AI assistant. How can I help you today?



In [18]:
response2 = chain_with_memory.invoke(
    {"name": "AI From Ollama", "user_input": "What is your name?"},
    config={"configurable": {"session_id": "user1"}}
)

print(response2.content)

I am Gemma, an open-weights AI model.



#### use open ai

In [62]:
openai_llm = ChatOpenAI(name="gpt-4o-mini", temperature=0.5, api_key=openai_api_key)

In [64]:
template_openai = ChatPromptTemplate(
    messages=[
        ("system","You are a helpful AI bot. Your name is {name}."),
        ("human","{user_input}")
    ]
)

In [65]:
openai_chain = template_openai | openai_llm

In [66]:
response = openai_chain.invoke({
    "name": "AI From Ollama",
    "user_input": "hello, what is your name?"
})

print(response.content)

Hello! My name is AI From Ollama. How can I assist you today?


#### now add memory to the chat

In [32]:
prompt_openai = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI bot. Your name is {name}."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{user_input}")
])

In [33]:
store_open_ai = {}
def get_session_history_openai(session_id: str):
    if session_id not in store_open_ai:
        store_open_ai[session_id] = ChatMessageHistory()
    return store_open_ai[session_id]

In [36]:
chain_with_memory_openai = RunnableWithMessageHistory(
    openai_chain,
    get_session_history_openai,
    input_messages_key="user_input",
    history_messages_key="history",
)

In [37]:
response1 = chain_with_memory_openai.invoke(
    {"name": "AI From Ollama", "user_input": "Hello!, my name is zkzk"},
    config={"configurable": {"session_id": "user1"}}
)

print(response1.content)


Hello zkzk! How can I assist you today?


In [38]:
response2 = chain_with_memory_openai.invoke(
    {"name": "AI From Ollama", "user_input": "What is your name?"},
    config={"configurable": {"session_id": "user1"}}
)

print(response2.content)

Hello! My name is AI From Ollama. How can I assist you today?


#### now what will use the model with tools 

In [68]:
@tool
def add(a , b):
    """"thos is a tool to add two numbers argument a and b and it will return the sum of a and b"""
    return a + b
openai_llm_with_tools = openai_llm.bind_tools([add])

In [72]:
response = openai_llm_with_tools.invoke([
    SystemMessage(content="You are a helpful AI bot. You have access to an add tool."),
    HumanMessage(content="Can you add 5 and 3 for me?")
])

print(response.tool_calls)

[{'name': 'add', 'args': {'a': 5, 'b': 3}, 'id': 'call_rItK4q9FW4HlNAZNuFVGfRek', 'type': 'tool_call'}]


In [76]:
if response.tool_calls:
    tool_call = response.tool_calls[0]

    tool_name = tool_call["name"]
    tool_args = tool_call["args"]
    tool_id = tool_call["id"]

    # Step 3: Execute tool manually
    if tool_name == "add":
        result = add.invoke(tool_args)
    print(f"Tool result: {result}")
    # Step 4: Send tool result back to model
    final_response = openai_llm_with_tools.invoke([
        SystemMessage(content="You are a helpful AI bot. You have access to an add tool."),
        HumanMessage(content="Can you add 5 and 3 for me?"),
        response,  # the AI tool call message
        ToolMessage(
            content=str(result),
            tool_call_id=tool_id
        )
    ])

    print(final_response.content)

Tool result: 8
The sum of 5 and 3 is 8.


### now to the main topic
#### Sequential Chains (Step-by-step pipeline)

Use when:
Output of one step becomes input of the next.
Example:
Step 1 → Generate a short topic explanation

Step 2 → Summarize it in 1 sentence

In [77]:
llm = ChatOllama(
    model="gemma3:270m",
    timeout=30,
    base_url="http://127.0.0.1:11434",
    use_mmap=True,
)

In [78]:
explain_prompt = ChatPromptTemplate.from_template(
    "Explain the following topic in detail:\n{topic}"
)

In [79]:
# Step 2 prompt
summarize_prompt = ChatPromptTemplate.from_template(
    "Summarize this in one sentence:\n{explanation}"
)

In [80]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

In [81]:
chain = (
    explain_prompt
    | llm # get the explanation
    | parser # parse the explanation to a string
    | (lambda explanation: {"explanation": explanation}) # convert the string to a dict with key explanation
    | summarize_prompt # use the explanation to get the summary
    | llm # get the summary
    | parser # parse the summary to a string
)

In [82]:
response = chain.invoke({
    "topic": "Artificial Intelligence"
})

In [84]:
print(response)

AI is a broad field encompassing computer learning, problem-solving, understanding, creativity, and automation. It's crucial to understand its potential, the challenges it faces, and the ethical considerations surrounding its development and deployment.



#### now use open ai

In [85]:
chain_openai = (
    explain_prompt
    | openai_llm # get the explanation
    | parser # parse the explanation to a string
    | (lambda explanation: {"explanation": explanation}) # convert the string to a dict with key explanation
    | summarize_prompt # use the explanation to get the summary
    | openai_llm # get the summary
    | parser # parse the summary to a string
)

In [86]:
response = chain_openai.invoke({
    "topic": "Artificial Intelligence"
})

In [88]:
print(response)

Artificial Intelligence is a branch of computer science focused on creating machines that can perform tasks requiring human intelligence, with different types and subfields such as machine learning, natural language processing, and robotics, used in various industries for applications like virtual assistants and medical diagnosis, while also raising ethical concerns regarding job displacement and bias in algorithms.


##### so the flow is like this 
``` code 
topic → explain → summarize → final output
```

#### now th Router Chains (Dynamic routing)
Use when:
You want to route user input to different prompts/models based on intent.
Example:
If math → math prompt
If writing → creative prompt

In [89]:
parser = StrOutputParser()

In [90]:
math_chain = (
    ChatPromptTemplate.from_template("Solve this math problem:\n{input}")
    | llm
    | parser
)

In [91]:
writing_chain = (
    ChatPromptTemplate.from_template("Write a short creative paragraph about:\n{input}")
    | llm
    | parser
)

In [92]:
from langchain_core.runnables.branch import RunnableBranch
router = RunnableBranch(
    (
        lambda x: "add" in x["input"] or "solve" in x["input"],
        math_chain,
    ),
    writing_chain,  # default
)

print(router.invoke({"input": "Solve 5 + 3"}))
print(router.invoke({"input": "Write about space exploration"}))

The answer to 5 + 3 is 8.

In the vast expanse of space, the universe whispers secrets untold, waiting to be discovered. From the glittering dust of a distant nebula to the icy scars of ancient asteroids, every celestial body holds the promise of new worlds and challenges to explore. Space exploration is more than just a pursuit of knowledge; it's a journey of hope, a quest to understand our place in the cosmos, and a constant reminder of the boundless wonders that await us beyond the stars.


#### same router but with open ai

In [93]:
math_chain = (
    ChatPromptTemplate.from_template("Solve this math problem:\n{input}")
    | openai_llm
    | parser
)

In [94]:
writing_chain = (
    ChatPromptTemplate.from_template("Write a short creative paragraph about:\n{input}")
    | openai_llm
    | parser
)

In [95]:
from langchain_core.runnables.branch import RunnableBranch
router = RunnableBranch(
    (
        lambda x: "add" in x["input"] or "solve" in x["input"],
        math_chain,
    ),
    writing_chain,  # default
)

print(router.invoke({"input": "Solve 5 + 3"}))
print(router.invoke({"input": "Write about space exploration"}))

As I sat pondering the simple math problem before me, the answer seemed to dance just out of reach. Five plus three. Eight, the logical part of my brain insisted. But as I stared at the numbers on the page, a wave of doubt washed over me. Could it be that simple? Was there a hidden complexity I was missing? With a deep breath, I tentatively wrote down my answer. Eight. And just like that, the puzzle was solved. The satisfaction of a problem conquered washed over me, leaving me with a renewed sense of confidence in my own abilities.
Space exploration has always captivated the human imagination, offering a glimpse into the vast unknown that lies beyond our planet. The idea of venturing into the infinite expanse of space, discovering new worlds and unraveling the mysteries of the universe, is both thrilling and awe-inspiring. From the first manned moon landing to the exploration of Mars and beyond, space exploration pushes the boundaries of human knowledge and technology. It is a testamen

##### flow of the code is 
``` code 
User input → condition → route → selected chain
```

#### now we will use Custom Chains with LCEL (LangChain Expression Language)
This is where it gets powerful.

LCEL lets you:
1-Parallelize
2-Transform
3-Merge outputs
4-Compose arbitrarily complex logic

In [97]:
from langchain_core.runnables import RunnableParallel , RunnableLambda

In [98]:
joke_chain = (
    ChatPromptTemplate.from_template("Tell a joke about {topic}")
    | llm
    | parser
)

serious_chain = (
    ChatPromptTemplate.from_template("Explain seriously what {topic} is")
    | llm
    | parser
)

parallel_chain = RunnableParallel(
    joke=joke_chain,
    serious=serious_chain
)

merge = RunnableLambda(
    lambda x: f"JOKE:\n{x['joke']}\n\nSERIOUS:\n{x['serious']}"
)

In [ ]:

final_chain = parallel_chain | merge

print(final_chain.invoke({"topic": "AI Agents"}))

JOKE:
Why did the AI agent get fired? 

Because it couldn't handle the challenge! 


SERIOUS:
AI Agents are a type of machine learning model that is designed to take a given task as input and perform a specific action or set of actions. They are essentially "thinking" and "learning" by interacting with data and using algorithms to make decisions.



#### now using openai

In [100]:
summarize_chain = (
    ChatPromptTemplate.from_template("summarize this {topic}")
    | openai_llm
    | parser
)

main_chain = (
    ChatPromptTemplate.from_template("Explain what {topic} is")
    | openai_llm
    | parser
)

parallel_chain = RunnableParallel(
    summarize=summarize_chain,
    main=main_chain
)

merge = RunnableLambda(
    lambda x: f"Summarize:\n{x['summarize']}\n\MAIN:\n{x['main']}"
)

In [101]:

final_chain = parallel_chain | merge

print(final_chain.invoke({"topic": "AI Agents"}))

Summarize:
AI agents are software programs that are designed to act autonomously, make decisions, and carry out tasks in a way that mimics human intelligence. These agents can gather information from their environment, analyze it, and make decisions based on that analysis. They can be used in a variety of applications, such as virtual assistants, autonomous vehicles, and video game characters. Overall, AI agents are a key component of artificial intelligence systems and play a crucial role in automating tasks and improving efficiency.
\MAIN:
AI agents are software programs or algorithms that have the ability to perceive their environment, make decisions, and take actions to achieve a specific goal. These agents use artificial intelligence techniques, such as machine learning and natural language processing, to analyze data, learn from past experiences, and adapt to changing circumstances. AI agents can be used in a wide range of applications, including autonomous vehicles, virtual assi